<a href="https://colab.research.google.com/github/lasentia/Analysis_of_card_dropout_rates_project/blob/main/2%ED%8C%80_2%EC%A1%B0/hyunalim/%EC%B9%B4%EC%9D%B4%EC%A0%9C%EA%B3%B1%EA%B2%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
#데이터 불러오고 필요없는 컬럼 삭
df = pd.read_csv('/content/BankChurners.csv')
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1, inplace=True)

df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [7]:
# 중복 데이터 확인
df.duplicated().sum()
len(df.CLIENTNUM.unique())

0

In [8]:
#결측치 확인
df.isnull().sum()

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

In [16]:
#범주형 변수 추출
category_cols = df.select_dtypes(include='O').columns.tolist()
category_cols

['Attrition_Flag',
 'Gender',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category']

###카이제곱 검정 - 범주형 변수와 이탈그룹의 상관관계 분석

In [15]:
#이탈한 고객데이터만 추출
attrited = df[df['Attrition_Flag'] == "Attrited Customer"]
attrited.reset_index(drop=True, inplace=True)
attrited.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,708508758,Attrited Customer,62,F,0,Graduate,Married,Less than $40K,Blue,49,...,3,3,1438.3,0,1438.3,1.047,692,16,0.600,0.000
1,708300483,Attrited Customer,66,F,0,Doctorate,Married,Unknown,Blue,56,...,4,3,7882.0,605,7277.0,1.052,704,16,0.143,0.077
2,779471883,Attrited Customer,54,F,1,Graduate,Married,Less than $40K,Blue,40,...,3,1,1438.3,808,630.3,0.997,705,19,0.900,0.562
3,714374133,Attrited Customer,56,M,2,Graduate,Married,$120K +,Blue,36,...,3,3,15769.0,0,15769.0,1.041,602,15,0.364,0.000
4,712030833,Attrited Customer,48,M,2,Graduate,Married,$60K - $80K,Silver,35,...,4,4,34516.0,0,34516.0,0.763,691,15,0.500,0.000


In [19]:
from scipy.stats import chi2_contingency

# 범주형 변수들을 선택
category_cols = df.select_dtypes(include='object').columns

# 가설 검정을 위한 함수
def chi2_test(column1, column2):
    contingency_table = pd.crosstab(df[column1], df[column2])
    chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
    return chi2_stat, p_value

# 결과를 저장할 빈 리스트 생성
results = []

# 각 범주형 변수 쌍에 대해 카이제곱 검정 수행
for i in range(len(category_cols)):
    for j in range(i+1, len(category_cols)):
        column1 = category_cols[i]
        column2 = category_cols[j]
        chi2_stat, p_value = chi2_test(column1, column2)
        results.append({'Variable 1': column1, 'Variable 2': column2, 'Chi2 Stat': chi2_stat, 'P-Value': p_value})

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
print(results_df)

         Variable 1       Variable 2    Chi2 Stat       P-Value
0    Attrition_Flag           Gender    13.865614  1.963585e-04
1    Attrition_Flag  Education_Level    12.511205  5.148913e-02
2    Attrition_Flag   Marital_Status     6.056097  1.089126e-01
3    Attrition_Flag  Income_Category    12.832259  2.500243e-02
4    Attrition_Flag    Card_Category     2.234227  5.252383e-01
5            Gender  Education_Level     7.288620  2.949785e-01
6            Gender   Marital_Status     3.875387  2.752383e-01
7            Gender  Income_Category  7138.449965  0.000000e+00
8            Gender    Card_Category    75.010351  3.604858e-16
9   Education_Level   Marital_Status    21.509010  2.545144e-01
10  Education_Level  Income_Category    45.253685  3.655347e-02
11  Education_Level    Card_Category    17.628879  4.803408e-01
12   Marital_Status  Income_Category    17.136251  3.107848e-01
13   Marital_Status    Card_Category    32.247110  1.804597e-04
14  Income_Category    Card_Category   1

In [29]:
# p-value가 0.05보다 작은 행 필터링
significant_results = results_df[results_df['P-Value'] < 0.05]

# 결과 출력
print("p-value가 0.05보다 작은 행:")
print(significant_results)

p-value가 0.05보다 작은 행:
         Variable 1       Variable 2    Chi2 Stat       P-Value
0    Attrition_Flag           Gender    13.865614  1.963585e-04
3    Attrition_Flag  Income_Category    12.832259  2.500243e-02
7            Gender  Income_Category  7138.449965  0.000000e+00
8            Gender    Card_Category    75.010351  3.604858e-16
10  Education_Level  Income_Category    45.253685  3.655347e-02
13   Marital_Status    Card_Category    32.247110  1.804597e-04
14  Income_Category    Card_Category   100.171346  1.210687e-14


### 수지형 데이터 범주형으로 변환하기
#### 'Months_on_book' 범주형 데이터로 변환 후 카이제곱검정 실행



In [39]:
df['Months_on_book'].describe()

count    10127.000000
mean        35.928409
std          7.986416
min         13.000000
25%         31.000000
50%         36.000000
75%         40.000000
max         56.000000
Name: Months_on_book, dtype: float64

In [46]:
import numpy as np

# 기존 데이터프레임에 새로운 범주형 변수 추가
df['Months_on_book_category'] = pd.cut(
    df['Months_on_book'],
    bins=[-np.inf, 12, 24, 36, 48, np.inf],
    labels=['1 year or less', '1-2 years', '2-3 years', '3-4 years', '4 years or more'],
    right=False
)

# 결과 확인
print(df[['Months_on_book', 'Months_on_book_category']].head())

   Months_on_book Months_on_book_category
0              39               3-4 years
1              44               3-4 years
2              36               3-4 years
3              34               2-3 years
4              21               1-2 years


In [47]:
from scipy.stats import chi2_contingency

# 'Months_on_book_category'과 'Attrition_Flag'의 교차 테이블 생성
contingency_table = pd.crosstab(df['Months_on_book_category'], df['Attrition_Flag'])

# 카이제곱 검정 수행
chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)

# 결과 출력
print("Chi2 Statistic:", chi2_stat)
print("P-Value:", p_value)

# 결과 해석
if p_value < 0.05:
    print("통계적으로 유의미한 상관관계가 있음")
else:
    print("상관관계가 유의미하지 않음")

Chi2 Statistic: 6.272287690637315
P-Value: 0.0990887277580675
상관관계가 유의미하지 않음
